In [1]:
import pandas as pd
from neo4j import GraphDatabase

In [7]:
# Read in biomedical benchmark (disease to drug explanations)
bmb = pd.read_csv('../data/biomedical_benchmark/drugmechdb_hetnet_disease_explanations.tsv', sep='\t')

In [6]:
# Initialize client for HetNet Neo4j
bolt = 'bolt://neo4j.het.io:7687'
driver = GraphDatabase.driver(bolt)
sess = driver.session()

In [55]:
def get_gene_counts(drug_id, disease_id, sess):
    q = f'''
    MATCH (c:Compound) WHERE c.identifier = '{drug_id}'
    MATCH (c)-[:BINDS_CbG]-(g1:Gene)
    WITH c, count(distinct g1) as drug_gene_ct
    MATCH (d:Disease) WHERE d.identifier='{disease_id}' 
    MATCH (d)-[:ASSOCIATES_DaG]-(g2:Gene)
    WITH c, drug_gene_ct, d, count(distinct g2) as dx_gene_ct
    MATCH p=(c)-[:BINDS_CbG]-(g3:Gene)-[:ASSOCIATES_DaG]-(d)
    RETURN c.name, drug_gene_ct, d.name, dx_gene_ct, count(distinct g3) as shared_gene_ct
    '''
    return sess.run(q)

In [58]:
result_rows = []
for i, row in bmb.iterrows():
    drug_id = row.drug_id_hetnet[10:]
    disease_id = row.outcome_id_hetnet[9:]
    out = get_gene_counts(drug_id, disease_id, sess)
    keys = out.keys()
    values = out.values()
    if len(values)==1:
        result = dict(zip(keys,values[0]))
        result_rows.append(result)
    else:
        print(f'skipping {i}')
all_counts = pd.DataFrame(result_rows)

skipping 2
skipping 3
skipping 11
skipping 26
skipping 27
skipping 29
skipping 31
skipping 37


In [59]:
all_counts

,c.name,drug_gene_ct,d.name,dx_gene_ct,shared_gene_ct
0,Ziprasidone,34,schizophrenia,235,15
1,Eletriptan,14,migraine,46,5
2,Venlafaxine,11,panic disorder,16,1
3,Amoxapine,23,endogenous depression,33,6
4,Citalopram,14,endogenous depression,33,3
5,Dexfenfluramine,9,obesity,373,3
6,Pimozide,23,Gilles de la Tourette syndrome,16,3
7,Prochlorperazine,7,schizophrenia,235,4
8,Amantadine,14,Parkinson's disease,143,5
9,Venlafaxine,11,panic disorder,16,1
